### Setup

In [1]:
import pandas as pd
import numpy as np
import os
from time import time

In [2]:
#import sys
#sys.path.append(os.path.abspath('..'))#sys.path[0] + '/..') 
from vb_estimators import  LinRegSupreme,LinSVR,RBFSVR,ENet,L1Lars,GBR,HGBR,FlexiblePipe
from vb_helper import VBHelper
from vb_cross_validator import regressor_q_stratified_cv

no daal4py


In [3]:

from warnings import filterwarnings
filterwarnings('ignore')


In [4]:
gridpoints=5
kwargs=dict(
    test_share=0.5,#for debugging. keep at zero otherwise
    cv_folds=5,
    cv_reps=3,
    #cv_groupcount=5,
    cv_strategy=('quantile',5), # for stratified cv
    random_state=2 # random_state for reproducibility
)
vbhelper=VBHelper(**kwargs)

In [5]:
scorer_list=['neg_mean_squared_error', 'neg_mean_absolute_error', 'r2'] #cross_validate wants strings
vbhelper.scorer_list=scorer_list

##### Example Dataset

In [6]:
data_path=os.path.join('sample_data','ex1.csv')
df=pd.read_csv(data_path)
all_vars=list(df.columns)
print(all_vars)

['STA_ID', 'LONG', 'LAT', 'OrigHabCode', 'Date', 'THG_Fish', 'YEAR', 'SEASON', 'SUBAREA', 'HABCODE', 'Floc_Depth_ft', 'AFDW_Floc', 'MEHG_Floc', 'THG_floc', 'Tot_Phos_floc', 'Bulk_Dens_Floc', 'Soil_Thickness_FT', 'AFDW_Soil', 'Bulk_Dens_Soil', 'PH_soil', 'SO4_soil', 'MEHG_soil', 'THG_soil', 'Tot_Carbon_Soil_%', 'Tot_Nitrogen_Soil_%', 'Tot_Phos_soil', 'Wat_Depth_ft', 'COND_SW', 'DO_SW', 'TEMP_SW', 'PH_SW', 'TURB_SW', 'REDOX_SW', 'Alk_Phos_SW', 'CHLA_SW', 'CL_SW', 'MEHG_SW', 'NH4_SW', 'NO2_SW', 'NO3_SW', 'SO4_SW', 'Sol_Reac_Phos_SW', 'THG_SW', 'TOC_SW', 'Tot_Nitrogen_SW', 'Tot_Phos_SW', 'REDOX_PW', 'H2S_PW', 'Sol_Reac_Phos_PW', 'MEHG_Peri_AVG', 'THG_epi_peri']


In [7]:
#The user sets the variables to use for x and y.
y_name='THG_Fish'
loc_vars=['LAT','LONG']
drop_vars=['Date','OrigHabCode','STA_ID']
drop_vars.extend(loc_vars)
drop_vars.append(y_name)
x_vars=[var for var in all_vars if var not in drop_vars]
X_df=df.loc[:,x_vars]
y_df=df.loc[:,y_name]

In [8]:

shuf=np.arange(y_df.shape[0])
np.random.seed(0)
np.random.shuffle(shuf)
X_df=X_df.iloc[shuf]
y_df=y_df.iloc[shuf]
vbhelper.setData(X_df,y_df)#no train/test split!
X_train, X_test, y_train, y_test=vbhelper.train_test_split() #just for debugging

In [9]:
print(X_train.shape,y_train.shape)

(387, 45) (387,)


In [10]:
inner_cv_dict={'cv_reps':1,'cv_folds':5,'cv_strategy':('quantile',5)}
inner_cv=vbhelper.getCV(cv_dict=inner_cv_dict)

prep_dict={'impute_strategy':'impute_knn5','cat_idx':vbhelper.cat_idx}

pipe_kwargs=dict(do_prep=False,prep_dict=prep_dict,inner_cv=inner_cv,gridpoints=gridpoints,cat_idx=vbhelper.cat_idx,float_idx=vbhelper.float_idx,bestT=False)
pipe_dict={
    #'gradient-boosting-reg':{'pipe':GBR,'pipe_kwargs':dict(
    #    prep_dict=prep_dict,do_prep=False)},
    #'lin-reg-supreme':{'pipe':LinRegSupreme,'pipe_kwargs':pipe_kwargs}, 
    #'powXB-least-sq':{'pipe':FlexiblePipe,'pipe_kwargs':{**pipe_kwargs,'flex_kwargs':{'form':'powXB'}}}),
    #'expXB-least-sq':{'pipe':FlexiblePipe,'pipe_kwargs':{**pipe_kwargs,'flex_kwargs':{'form':'expXB'}}}), #expXB is default
    #'nonlinear-search-least-sq': {'pipe':FlexiblePipe,'pipe_kwargs':{**pipe_kwargs,'functional_form_search':True}}},
    #'robust-powXB-least-sq':{'pipe':FlexiblePipe,'pipe_kwargs':{**pipe_kwargs,'flex_kwargs':{'form':'powXB','robust':True}}},
    #'robust-expXB-least-sq':{'pipe':FlexiblePipe,'pipe_kwargs':{**pipe_kwargs,'flex_kwargs':{'form':'expXB','robust':True}}}, #expXB is default
    #'robust-nonlinear-search-least-sq': {'pipe':FlexiblePipe,'pipe_kwargs':{**pipe_kwargs,'functional_form_search':True,'flex_kwargs':{'robust':True}}},
    #'histogram-gradient-boosting-reg':{'pipe':HGBR,'pipe_kwargs':{}}
    
    
    #'elastic-net':{'pipe':ENet,'pipe_kwargs':pipe_kwargs}, 
    #'linear-svr-cv':{'pipe':LinSVR,'pipe_kwargs':pipe_kwargs}
    'rbf-svr-cv':{'pipe':RBFSVR,'pipe_kwargs':pipe_kwargs}, 
    'lassolars':{'pipe':L1Lars,'pipe_kwargs':pipe_kwargs},
    }


#estimator_dict={'multi_pipe':{'pipe':MultiPipe(pipelist=[(k,v) for k,v in estimator_dict.items()],cat_idx=vbhelper.cat_idx)}
vbhelper.setEstimatorDict(pipe_dict)
vbhelper.setModelDict()
#vbhelper.model_dict={key:val() for key,val in vbhelper.estimator_dict.items()} # they will be models once .fit is called

In [11]:
'''if False:    
    pipe_kwargs2=pipe_kwargs
    pipe_kwargs2['do_prep']=True
    name='lone-lasso'
    vbhelper.estimator_dict[name]={'pipe':L1Lars,'pipe_kwargs':pipe_kwargs2}
    vbhelper.model_dict[name]=vbhelper.estimator_dict[name]()'''

"if False:    \n    pipe_kwargs2=pipe_kwargs\n    pipe_kwargs2['do_prep']=True\n    name='lone-lasso'\n    vbhelper.estimator_dict[name]={'pipe':L1Lars,'pipe_kwargs':pipe_kwargs2}\n    vbhelper.model_dict[name]=vbhelper.estimator_dict[name]()"

In [12]:
i=0
for name,est in vbhelper.model_dict.items():
    start=time()
    i+=1;print(name)
    est.fit(X_train,y_train)
    print('train R2:',est.score(X_train,y_train))
    if not X_test is None:print('test R2:',est.score(X_test,y_test))
    end=time()
    print(f'runtime:{(end-start)/60} min.\n')


multi_pipe
train R2: 0.5977812261622973
test R2: 0.42313341251432135
runtime:0.18532989819844564 min.



In [13]:
mp=vbhelper.model_dict['multi_pipe']
fitted_ipipe_dict=mp.build_individual_fitted_pipelines()
fitted_ipipe_dict['lassolars'].score(X_test,y_test)

0.3794293618369128

In [14]:
#

In [15]:
#

In [16]:
vbhelper.runCrossValidate()

multi_pipe,[('neg_mean_squared_error', nan), ('neg_mean_absolute_error', nan), ('r2', nan)], runtime:0.06464209953943888 min.
MultiPipe(pipelist=[('rbf-svr-cv',
                     {'pipe': <class 'vb_estimators.RBFSVR'>,
                      'pipe_kwargs': {'bestT': False, 'cat_idx': (0, 1, 2, 3),
                                      'do_prep': False,
                                      'float_idx': [4, 5, 6, 7, 8, 9, 10, 11,
                                                    12, 13, 14, 15, 16, 17, 18,
                                                    19, 20, 21, 22, 23, 24, 25,
                                                    26, 27, 28, 29, 30, 31, 32,
                                                    33, ...],
                                      'gridpoints': 5,
                                      'inner_cv': <vb_cross_validator.regressor_q_stratified_cv object at 0x7f6c34c3c7f0>,
                                      'prep_dict': {'ca...
                      'pi

AttributeError: 'StackingRegressor' object has no attribute 'named_estimators_'

In [ ]:
cv=vbhelper.getCV()
for train_idx,test_idx in cv.split(X_train,y_train):
    for name,pipe in fitted_ipipe_dict.items():
        print(pipe.score(X_test,y_test))
        assert False,'pause'

In [ ]:
#plot cv_yhat

#### graphs and table to summarize results

In [ ]:
vbhelper.buildCVScoreDict()

In [ ]:
vbhelper.viewCVScoreDict()

In [ ]:
vbhelper.plotCVScores(sort=1)

In [ ]:
vbhelper.predictCVYhat()

In [ ]:
vbhelper.plotCVYhat(single_plot=True)